In [1]:
#from keys import ml_prod_servicekey as ml_key
from keys import ml_servicekey_hole, ml_servicekey_scratch
from ml_interface import MLFoundationClient, DefectCode
from database_interface import DBClient
from image_interface import ImageDBClient

In [3]:
# init the clients
db_client = DBClient()
#ml_client = MLFoundationClient(ml_key, offline = True)
im_client = ImageDBClient()

In [4]:
img_path = "test-photo2.jpeg"
img_path2 = "Hole.jpg"
img = open(img_path, "rb")
img2 = open(img_path, "rb")

In [5]:
# use this key to upload to the database
key = im_client.upload(img_path)
key2 = im_client.upload(img_path2)

In [6]:

# for every model
clientHole = MLFoundationClient(ml_servicekey_hole)
defectHole = clientHole.modelPredictOne(img_path, "hole","HoleModel", DefectCode.Hole)
defectHole2 = clientHole.modelPredictOne(img_path2, "hole","HoleModel", DefectCode.Hole)

clientScratch = MLFoundationClient(ml_servicekey_scratch)
defectScratch = clientScratch.modelPredictOne(img_path, "scratch","ScratchModel", DefectCode.Scratch)
defectScratch2 = clientScratch.modelPredictOne(img_path2, "scratch","ScratchModel", DefectCode.Scratch)

defects = list(set(defectHole+defectHole2+defectScratch+defectScratch2))

#unite the lists


#make the json
imageInfo= {
    'image1' : key,
    'image2' : key2,
    'defects' : defects     
}
print(imageInfo)

{'image1': '1d60ec14-710e-4f56-96b1-36320f7dd1d1', 'image2': '46c18248-ea30-48de-a0b1-8c8db038d06e', 'defects': [<DefectCode.Scratch: 2>, <DefectCode.Hole: 5>]}


In [7]:
db_client.saveMachineLearningResult(imageInfo, productExists=False)

New register inserted succesfully


True